In [6]:
!!pip install litellm

import os
from google.colab import userdata
api_key = userdata.get('OPENAI_API_KEY')
os.environ['OPENAI_API_KEY'] = api_key

In [7]:
from litellm import completion
from typing import List, Dict

def generate_response(messages: List[Dict]) -> str:
    """Call LLM to get response"""
    response = completion(
        model="openai/gpt-4o",
        messages=messages,
        max_tokens=1024
    )
    return response.choices[0].message.content

In [8]:
def extract_code_block(response: str) -> str:
    """Extract code block from response"""
    if not '```' in response:
        return response

    code_block = response.split('```')[1].strip()
    if code_block.startswith("python"):
        code_block = code_block[6:]

    return code_block

In [9]:
print("\nWhat kind of function would you like to create?")
print("Example: 'A function that calculates the factorial of a number'")
print("Your description: ", end='')
function_description = input().strip()


What kind of function would you like to create?
Example: 'A function that calculates the factorial of a number'
Your description: Calculate fibonacci sequence up to n


In [10]:
# Initialize conversation with system prompt
messages = [
      {"role": "system", "content": "You are a Python expert helping to develop a function."}
   ]

In [11]:
# First prompt - Basic function
messages.append({
      "role": "user",
      "content": f"Write a Python function that {function_description}. Output the function in a ```python code block```."
   })

initial_function = generate_response(messages)

In [12]:
# Parse the response to get the function code
initial_function = extract_code_block(initial_function)

print("\n=== Initial Function ===")
print(initial_function)


=== Initial Function ===

def fibonacci_sequence(n):
    """
    Calculate the Fibonacci sequence up to the nth term.
    
    Parameters:
    n (int): The number of terms to calculate in the Fibonacci sequence.

    Returns:
    list: A list containing the Fibonacci sequence up to the nth term.
    """
    if n <= 0:
        return []
    elif n == 1:
        return [0]
    elif n == 2:
        return [0, 1]

    fib_sequence = [0, 1]
    for i in range(2, n):
        next_value = fib_sequence[-1] + fib_sequence[-2]
        fib_sequence.append(next_value)
    
    return fib_sequence

# Example usage:
# fibonacci_sequence(10) will return [0, 1, 1, 2, 3, 5, 8, 13, 21, 34]


In [13]:
# Add assistant's response to conversation
# Notice that I am purposely causing it to forget its commentary and just see the code so that
# it appears that is always outputting just code.
messages.append({"role": "assistant", "content": "\`\`\`python\n\n"+initial_function+"\n\n\`\`\`"})

In [14]:
# Second prompt - Add documentation
messages.append({
      "role": "user",
      "content": "Add comprehensive documentation to this function, including description, parameters, "
                 "return value, examples, and edge cases. Output the function in a ```python code block```."
   })

In [15]:
documented_function = generate_response(messages)
documented_function = extract_code_block(documented_function)

print("\n=== Documented Function ===")
print(documented_function)


=== Documented Function ===

def fibonacci_sequence(n):
    """
    Calculate the Fibonacci sequence up to the nth term.
    
    The Fibonacci sequence is a series of numbers where each number is the sum 
    of the two preceding ones, typically starting with 0 and 1. That is, the 
    sequence goes 0, 1, 1, 2, 3, 5, 8, and so on.

    Parameters:
    n (int): The number of terms to calculate in the Fibonacci sequence.
             Must be a non-negative integer.

    Returns:
    list: A list containing the Fibonacci sequence up to the nth term.

    Examples:
    --------
    >>> fibonacci_sequence(0)
    []

    >>> fibonacci_sequence(1)
    [0]

    >>> fibonacci_sequence(2)
    [0, 1]

    >>> fibonacci_sequence(5)
    [0, 1, 1, 2, 3]

    >>> fibonacci_sequence(10)
    [0, 1, 1, 2, 3, 5, 8, 13, 21, 34]

    Edge Cases:
    -----------
    - If n is 0, return an empty list since no terms are requested.
    - If n is 1, return [0] as it is the first Fibonacci number.
    - Negati

In [16]:
# Add documentation response to conversation
messages.append({"role": "assistant", "content": "\`\`\`python\n\n"+documented_function+"\n\n\`\`\`"})

# Third prompt - Add test cases
messages.append({
      "role": "user",
      "content": "Add unittest test cases for this function, including tests for basic functionality, "
                 "edge cases, error cases, and various input scenarios. Output the code in a \`\`\`python code block\`\`\`."
   })

test_cases = generate_response(messages)

In [17]:
# We will likely run into random problems here depending on if it outputs JUST the test cases or the
# test cases AND the code. This is the type of issue we will learn to work through with agents in the course.
test_cases = extract_code_block(test_cases)
print("\n=== Test Cases ===")
print(test_cases)


=== Test Cases ===

import unittest

def fibonacci_sequence(n):
    """
    Calculate the Fibonacci sequence up to the nth term.
    """
    if n <= 0:
        return []
    elif n == 1:
        return [0]
    elif n == 2:
        return [0, 1]

    fib_sequence = [0, 1]
    for i in range(2, n):
        next_value = fib_sequence[-1] + fib_sequence[-2]
        fib_sequence.append(next_value)
    
    return fib_sequence

class TestFibonacciSequence(unittest.TestCase):
    def test_basic_functionality(self):
        # Basic functionality tests
        self.assertEqual(fibonacci_sequence(5), [0, 1, 1, 2, 3])
        self.assertEqual(fibonacci_sequence(10), [0, 1, 1, 2, 3, 5, 8, 13, 21, 34])

    def test_edge_cases(self):
        # Edge cases
        self.assertEqual(fibonacci_sequence(0), [])
        self.assertEqual(fibonacci_sequence(1), [0])
        self.assertEqual(fibonacci_sequence(2), [0, 1])

    def test_validity_for_negative_input(self):
        # Test for negative input, sho

In [18]:
# Generate filename from function description
filename = function_description.lower()
filename = ''.join(c for c in filename if c.isalnum() or c.isspace())
filename = filename.replace(' ', '_')[:30] + '.py'

   # Save final version
with open(filename, 'w') as f:
  f.write(documented_function + '\n\n' + test_cases)


In [19]:
print("\n=== Initial Function ===")
print(initial_function)

print("\n=== Documented Function ===")
print(documented_function)

print("\n=== Test Cases ===")
print(test_cases)


=== Initial Function ===

def fibonacci_sequence(n):
    """
    Calculate the Fibonacci sequence up to the nth term.
    
    Parameters:
    n (int): The number of terms to calculate in the Fibonacci sequence.

    Returns:
    list: A list containing the Fibonacci sequence up to the nth term.
    """
    if n <= 0:
        return []
    elif n == 1:
        return [0]
    elif n == 2:
        return [0, 1]

    fib_sequence = [0, 1]
    for i in range(2, n):
        next_value = fib_sequence[-1] + fib_sequence[-2]
        fib_sequence.append(next_value)
    
    return fib_sequence

# Example usage:
# fibonacci_sequence(10) will return [0, 1, 1, 2, 3, 5, 8, 13, 21, 34]

=== Documented Function ===

def fibonacci_sequence(n):
    """
    Calculate the Fibonacci sequence up to the nth term.
    
    The Fibonacci sequence is a series of numbers where each number is the sum 
    of the two preceding ones, typically starting with 0 and 1. That is, the 
    sequence goes 0, 1, 1, 2, 3, 